### 현재 사용 안함<br>8개 열 기준

In [1]:
import pandas as pd

In [2]:
!pip install lightgbm==4.1.0
!pip install xgboost==2.0.0
!pip install sklearn
!pip install scikit-optimize

In [3]:
# 데이터 불러오기
diabate = pd.read_csv('diabates_full.csv', index_col=0)
diabate

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,125.0,33.6,0.627,50,1
1,1,85.0,66.0,29.0,125.0,26.6,0.351,31,0
2,8,183.0,64.0,29.0,125.0,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101.0,76.0,48.0,180.0,32.9,0.171,63,0
764,2,122.0,70.0,27.0,125.0,36.8,0.340,27,0
765,5,121.0,72.0,23.0,112.0,26.2,0.245,30,0
766,1,126.0,60.0,29.0,125.0,30.1,0.349,47,1


In [4]:
diabate_ = diabate.copy()

In [5]:
# 정답 데이터 분리
X = diabate_.drop(columns=['Outcome'])
y = diabate_[['Outcome']]

In [6]:
# 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=0.3, random_state=0)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

numeric_columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

# RobustScaler 적용
preprocessor = ColumnTransformer(
    transformers=[('numeric', RobustScaler(), numeric_columns)]
)

In [8]:
preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)]) # preprocessing-only

preprocessor_pipe.fit(X_train)

x_train_transformed = preprocessor_pipe.transform(X_train)
x_test_transformed = preprocessor_pipe.transform(X_test)

pd.DataFrame(x_train_transformed).head(3)

,0,1,2,3,4,5,6,7
0,-0.6,0.761905,1.256773,2.125,0.0,1.077778,-0.018767,-0.470588
1,-0.4,-0.857143,-0.314656,0.000,0.0,-1.577778,0.659517,-0.117647
2,-0.2,0.071429,-0.171799,-0.250,0.0,0.488889,-0.101877,-0.117647


In [9]:
# 모델 학습 및 하이퍼파라미터 최적화를 위한 필요한 라이브러리
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from skopt import BayesSearchCV
from collections import OrderedDict


In [10]:
# %%time
# # lightGBM
# # HPO - Baysian Search

# from skopt.space import Real, Integer # 실수, 정수형 변수 사용

# # Hyper-parameter 검색범위 지정
# param_grid = {'n_estimators': Integer(1000, 2000),
#               'learning_rate': Real(0.01, 0.1),
#               'num_leaves': Integer(1000, 3000),
#               'max_depth': Integer(-1, 5),
#               'scale_pos_weight': Real(1.2, 1.5)}

# # 최적의 하이퍼파라미터 찾기
# hpo_lgbm = BayesSearchCV(LGBMClassifier(random_state=42), param_grid,
#                     scoring='recall', refit=True,
#                     n_jobs=-1,
#                     n_iter=72,
#                     cv=5) # 평가지표 Recall 지정

# hpo_lgbm.fit(x_train_transformed, y_train)


# print('The best parameters are ', hpo_lgbm.best_params_)

The best parameters are  OrderedDict([('learning_rate', 0.01), ('max_depth', 1), ('n_estimators', 1000), ('num_leaves', 3000), ('scale_pos_weight', 1.5)])

In [20]:
# 최적의 하이퍼파라미터 추출
best_params = OrderedDict([('learning_rate', 0.01), ('max_depth', 1), ('n_estimators', 1000), ('num_leaves', 3000), ('scale_pos_weight', 1.5)])

model_lgbm = LGBMClassifier(n_estimators=best_params['n_estimators'],
                            learning_rate=best_params['learning_rate'],
                            max_depth=best_params['max_depth'],
                            num_leaves=best_params['num_leaves'],
                            scale_pos_weight=best_params['scale_pos_weight'],
                            class_weight='balanced',
                            boost_from_average=False,
                            verbose=-1)

model_lgbm.fit(x_train_transformed, y_train.values.ravel())

pred = model_lgbm.predict(x_test_transformed)
pred_proba = model_lgbm.predict_proba(x_test_transformed)[:, 1]

confusion_without = confusion_matrix(y_test, pred)
accuracy_without = accuracy_score(y_test, pred)
precision_without = precision_score(y_test, pred)
recall_without = recall_score(y_test, pred)
f1_without = f1_score(y_test, pred)
roc_auc_without = roc_auc_score(y_test, pred_proba)

print('Confusion matrix:')
print(confusion_without)
print()

print('Accuracy :', accuracy_without)
print('Precision :', precision_without)
print('Recall :', recall_without)
print('F1 :', f1_without)
print('ROC_AUC :', roc_auc_without)


Confusion matrix:

Accuracy : 0.7748917748917749
Precision : 0.6057692307692307
Recall : 0.8513513513513513
F1 : 0.7078651685393257
ROC_AUC : 0.8333620244448271


In [12]:
# %%time

# # XGBoost
# # HPO - Baysian Search
# from skopt.space import Real, Integer

# param_grid_xgb = {'n_estimators': Integer(1000, 2000),
#                   'learning_rate': Real(0.01, 0.1),
#                   'min_child_weight': Integer(2, 10),
#                   'max_depth': Integer(2, 4),
#                   'scale_pos_weight': Real(1.2, 1.5),
#                   }

# hpo_xgb = BayesSearchCV(xgb.XGBClassifier(random_state=42), param_grid_xgb, scoring='recall',
#                         refit=True, n_jobs=-1, n_iter=72, cv=5)

# hpo_xgb.fit(x_train_transformed, y_train)

# # 최적의 Hyper-parameter 출력
# print('The best parameters are ', hpo_xgb.best_params_)

The best parameters are  OrderedDict([('learning_rate', 0.01), ('max_depth', 2), ('min_child_weight', 8), ('n_estimators', 1000), ('scale_pos_weight', 1.5)])




In [19]:
# 최적의 하이퍼파라미터 추출
best_params = OrderedDict([('learning_rate', 0.01), ('max_depth', 2), ('min_child_weight', 8), ('n_estimators', 1000), ('scale_pos_weight', 1.5)])

model_xgb = xgb.XGBClassifier(n_estimators=best_params['n_estimators'],
                                  learning_rate= best_params['learning_rate'],
                                  max_depth = best_params['max_depth'],
                                  min_child_weight=best_params['min_child_weight'],
                                  scale_pos_weight=best_params['scale_pos_weight'],
                                  use_label_encoder=False,
                                  eval_metric='logloss',
                                  verbosity=1)

model_xgb.fit(x_train_transformed, y_train.values.ravel())

pred = model_xgb.predict(x_test_transformed)
pred_proba = model_xgb.predict_proba(x_test_transformed)[:, 1]

confusion_without = confusion_matrix(y_test, pred)
accuracy_without = accuracy_score(y_test, pred)
precision_without = precision_score(y_test, pred)
recall_without = recall_score(y_test, pred)
f1_without = f1_score(y_test, pred)
roc_auc_without = roc_auc_score(y_test, pred_proba)

print('Confusion matrix:')
print(confusion_without)
print()

print('Accuracy :', accuracy_without)
print('Precision :', precision_without)
print('Recall :', recall_without)
print('F1 :', f1_without)
print('ROC_AUC :', roc_auc_without)

Confusion matrix:
confusion_without

Accuracy : 0.7748917748917749
Precision : 0.6486486486486487
Recall : 0.6486486486486487
F1 : 0.6486486486486487
ROC_AUC : 0.8253572043380961


In [14]:
sample_data = pd.DataFrame([['9', '145', '88', '34', '165', '30.3', '0.771', '5']],
                           columns=['Pregnancies', 'Glucose', 'BloodPressure',
                                    'SkinThickness','Insulin','BMI',
                                    'DiabetesPedigreeFunction','Age']).astype(float)

model_lgbm.predict(sample_data)

array([1])

In [15]:
model_xgb.predict(sample_data)

array([1])

In [16]:
# import joblib

# # Save model
# joblib.dump(model_lgbm, 'ml_lgbm.pkl')

# # Load saved model
# # loaded_model_lgbm = joblib.load('model_lgbm.pkl')


In [17]:
# import pickle

# # Save model
# with open('ml_xgb.pkl', 'wb') as f:
#     pickle.dump(model_xgb, f)


In [18]:


# # 위 2개 모델 새로이 학습 / 모델 파일로 저장 / 성능 기록 / 스크린샷 촬영 (3가지 모델 성능 비교 준비 @ ppt)

# x_test_df = pd.DataFrame(x_test_transformed, columns=['Pregnancies','Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',	'BMI', 'DiabetesPedigreeFunction', 'Age'])
# x_test_df.to_csv('x_full_test_df.csv')
# y_test_df = pd.DataFrame(y_test)
# y_test_df.to_csv('y_full_test_df.csv')
# # x_test_df -> 파일로 저장 -> colab 에서 해당 데이터프레임으로 아래 예측 진행 & x_test_df 기준 acc 체크 후 공유

# # 3가지 모델 중 가장 좋은 acc 모델 찾아 상휘님께 공유



# # import joblib
# # import numpy as np

# # model_loaded = joblib.load('final_blended_3.pkl')

# # x_test_df = pd.DataFrame(x_test_transformed, columns=['Glucose', 'BMI', 'age'])

# # pred = model_loaded.predict(x_test_df)
# # pred_proba = model_loaded.predict_proba(x_test_df)[:, 1]

# # confusion_without = confusion_matrix(y_test, pred)
# # accuracy_without = accuracy_score(y_test, pred)
# # precision_without = precision_score(y_test, pred)
# # recall_without = recall_score(y_test, pred)
# # f1_without = f1_score(y_test, pred)
# # roc_auc_without = roc_auc_score(y_test, pred_proba)

# # print('Confusion matrix:')
# # print(confusion_without)
# # print()

# # print('Accuracy :', accuracy_without)
# # print('Precision :', precision_without)
# # print('Recall :', recall_without)
# # print('F1 :', f1_without)
# # print('ROC_AUC :', roc_auc_without)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=645823d6-5f8a-49f6-b0db-26e2163dbcc0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>